## Burned area

In [1]:
workflow = dict([('id', 'burned-area'),
                ('label', 'Burned area delineation'),
                ('doc', 'Burned area delineation using two techniques')])


In [2]:
pre_event = dict([('id', 'pre_event'), 
                  ('label', 'Pre-event product for burned area delineation'),
                  ('doc', 'Pre-event product for burned area delineation'),
                  ('value', '/workspace/application-chaining/jn0iisx4/stac-results'), 
                  ('type', 'Directory')])

In [3]:
post_event = dict([('id', 'post_event'), 
                  ('label', 'Post-event product for burned area delineation'),
                  ('doc', 'Post-event product for burned area delineation'),
                  ('value', '/workspace/application-chaining/o27byuyx/stac-results'), 
                  ('type', 'Directory')])

In [4]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('label', 'NDVI difference threshold'),
                       ('doc', 'NDVI difference threshold'),
                       ('type', 'string')]) 

In [5]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('label', 'NDWI difference threshold'),
                       ('doc', 'NDWI difference threshold'),
                       ('type', 'string')]) 

In [7]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import Catalog, Collection, EOItem, MediaType, EOAsset, CatalogType
from time import sleep

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_nbr/'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
pre_event_cat = Catalog.from_file(os.path.join(pre_event['value'], 'catalog.json'))
post_event_cat = Catalog.from_file(os.path.join(post_event['value'], 'catalog.json'))

In [10]:
pre_event_item = next(pre_event_cat.get_items())

pre_event_item.assets

{'nbr': <EOAsset href=./NBR_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif>,
 'ndvi': <EOAsset href=./NDVI_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif>,
 'ndwi': <EOAsset href=./NDWI_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif>}

In [11]:
post_event_item = next(post_event_cat.get_items())

post_event_item.assets

{'nbr': <EOAsset href=./NBR_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>,
 'ndvi': <EOAsset href=./NDVI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>,
 'ndwi': <EOAsset href=./NDWI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>}

In [13]:
scaling_factor = 1/10000

scaling_factor

0.0001

If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

In [14]:
_mem = '/vsimem/mem.tif'

Process NDVI difference

In [15]:
temp_ds = gdal.Translate(_mem,
                         pre_event_item.assets['ndvi'].get_absolute_href(),
                         outputType=gdal.GDT_Int16)

width = temp_ds.RasterXSize
height = temp_ds.RasterYSize
geo_transform = temp_ds.GetGeoTransform()
geo_ref = temp_ds.GetProjectionRef()

pre_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [16]:
temp_ds = gdal.Translate(_mem,
                         post_event_item.assets['ndvi'].get_absolute_href(),
                         outputType=gdal.GDT_Int16)

post_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [17]:
delta_ndvi = ((pre_ndvi - post_ndvi) * scaling_factor).astype(float)

pre_ndvi = post_ndvi = None

Process NDWI difference

In [18]:
temp_ds = gdal.Translate(_mem,
                         pre_event_item.assets['ndwi'].get_absolute_href(),
                         outputType=gdal.GDT_Int16)

pre_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [19]:
temp_ds = gdal.Translate(_mem,
                         post_event_item.assets['ndwi'].get_absolute_href(),
                         outputType=gdal.GDT_Int16)

post_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [20]:
delta_ndwi = ((pre_ndwi - post_ndwi) * scaling_factor).astype(float)

pre_ndwi = post_ndwi = None

Burned area delineation

In [21]:
burned = np.where(((delta_ndwi  > float(ndwi_threshold['value'])) & (delta_ndvi > float(ndvi_threshold['value']))), 1, 0) 

In [23]:
def cog(input_tif, output_tif,no_data=None):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                    '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                    '-co COMPRESS=LZW '))
    
    if no_data != None:
        translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                        '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                        '-co COMPRESS=LZW '\
                                                                        '-a_nodata {}'.format(no_data)))
    ds = gdal.Open(input_tif, gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    
    ds = None

    ds = gdal.Open(input_tif)
    gdal.Translate(output_tif,
                   ds, 
                   options=translate_options)
    ds = None

    os.remove('{}.ovr'.format(input_tif))
    os.remove(input_tif)


In [25]:
output_name = 'DELINEATION'

In [26]:
bands = [{'name': 'VI_THRESHOLD',
          'common_name': ''}, 
         {'name': 'RBR',
          'common_name': ''}
        ]

In [28]:
temp_name = '_BURNED_NDVI_NDWI_THRESHOLD.tif'
output_name = 'BURNED_NDVI_NDWI_THRESHOLD.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Byte)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(burned),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)

relativized burn ratio (RBR) 

In [29]:
temp_ds = gdal.Translate(_mem,
                         pre_event_item.assets['nbr'].get_absolute_href(),
                         outputType=gdal.GDT_Int16)

pre_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [30]:
temp_ds = gdal.Translate(_mem,
                         post_event_item.assets['nbr'].get_absolute_href(),
                         outputType=gdal.GDT_Int16)

post_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [31]:
delta_nbr = ((pre_nbr  - post_nbr) * scaling_factor).astype(float)

post_nbr = None

In [32]:
rbr = delta_nbr / (pre_nbr * scaling_factor + 1.001)

delta_nbr = pre_nbr = None

In [33]:
temp_name = '_BURNED_RBR.tif'
output_name = 'BURNED_RBR.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(rbr),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)